# Run all files

## Generation of accumulated point cloud

In [7]:
"""
Load dataset
"""

import os
import time

start_time = time.time()

data_path = os.path.abspath("parsed_kitti_data_181212.pickle")


import pickle

with open(data_path, 'rb') as f:
    parsed_kitti_data = pickle.load(f)

print "Dataset was loaded"

"""
Data structure
"""

for date in parsed_kitti_data:
    print "date:", date
    
    for drive in parsed_kitti_data[date]:
        print "\tdrive:", drive

IOError: [Errno 2] No such file or directory: '/media/kyungpyo/SmartCarContest/MasterThesisProject/KITTI_Evaluation/notebook/parsed_kitti_data_181212.pickle'

In [2]:
import copy
import pickle


parsed_kitti_data_w_r = {}

for date in parsed_kitti_data:
    print "date:", date
    parsed_kitti_data_w_r[date] = {}
    
    for drive in parsed_kitti_data[date]:
                
        print "\tdrive:", drive
        
        parsed_kitti_data_w_r_drive = copy.deepcopy(parsed_kitti_data_w_r)
        
        parsed_kitti_data_w_r_drive[date][drive] = []
        
        for tracklets_list in parsed_kitti_data[date][drive]:
            
            tracklets_list_w_r = {}
            
            print "\t\ttrack: ",
            for key in tracklets_list:
                
                print " ", key,
                tracklets_list_w_r[key] = tracklets_list[key]
                
            print "{}".format(len(tracklets_list[key]))
            
            parsed_kitti_data_w_r_drive[date][drive].append(tracklets_list_w_r)
            
        """ 
        Save pickle
        """
        out_file_nme = "parsed_kitti_data_181212_{}.pickle".format(drive)
        with open(out_file_nme, 'wb') as f:
            pickle.dump(parsed_kitti_data_w_r_drive, f, protocol=2)

        print("\n\nSaved new dataset, protocol=2, {}\n\n".format(out_file_nme))
        del parsed_kitti_data_w_r_drive
            

print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"

date: 2011_09_26
	drive: 0002
		track:    rot   trans   type   rect   pc 11
		track:    rot   trans   type   rect   pc 28
		track:    rot   trans   type   rect   pc 16


Saved new dataset, protocol=2, parsed_kitti_data_181212_0002.pickle




Execution time: 0.107073068619 




## Registration

In [3]:
import re

"""
Search all files 
"""
basedir = './'
p = re.compile('parsed_kitti_data_181212_\d{4}')

drives = []

cnt = 0
for root,d_names,f_names in os.walk(basedir):
    for f in f_names:
        m = p.match(f)
        if m:
            cnt +=1
            print 'Match found: ', m.group(), cnt
            drives.append(m.group() + ".pickle")

Match found:  parsed_kitti_data_181212_0002 1


In [4]:
import tqdm
import sys
import pickle


from registration import *


for d in drives:
    
    """
    Load dataset
    """
    with open(d, 'rb') as f:
        parsed_kitti_data_w_r = pickle.load(f)
    

    for date in parsed_kitti_data_w_r:
        print "date:", date
        
        assert len(parsed_kitti_data_w_r[date]) == 1

        for drive in parsed_kitti_data_w_r[date]:

            print "\tdrive:", drive

            sys.stdout.flush()

            for i in tqdm.trange( len(parsed_kitti_data_w_r[date][drive]) ):

                pc = parsed_kitti_data_w_r[date][drive][i]['pc']

                """
                Registration
                """
                parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = []
                sys.stdout.flush()
                if len(pc) > 1:
                    parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = registration_layered(pc)
                else:
                    parsed_kitti_data_w_r[date][drive][i]['pc_wr'] = []

                    print "\tdrive:", drive
                    
    """
    Save data
    """
    out_data_path = 'parsed_kitti_data_w_r_181212_{}.pickle'.format(drive)
    
    with open(out_data_path, 'wb') as f:
        pickle.dump(parsed_kitti_data_w_r, f, protocol=2)
    
    print("\n\nSaved new dataset, protocol=2")
    
print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"

date: 2011_09_26
	drive: 0002


  0%|          | 0/3 [00:00<?, ?it/s]

11
28


 67%|██████▋   | 2/3 [00:16<00:08,  8.16s/it]

16


100%|██████████| 3/3 [00:24<00:00,  8.13s/it]



Saved new dataset, protocol=2


Execution time: 24.9234850407 




## Classification

In [5]:
import re

"""
Search all files 
"""
basedir = './'
p = re.compile('parsed_kitti_data_w_r_181212_\d{4}')

drives = []

cnt = 0
for root,d_names,f_names in os.walk(basedir):
    for f in f_names:
        m = p.match(f)
        if m:
            cnt +=1
            print 'Match found: ', m.group(), cnt
            drives.append(m.group() + ".pickle")

Match found:  parsed_kitti_data_w_r_181212_0002 1


In [6]:
import tqdm
import sys
import pickle
import os, sys
import tensorflow as tf

from registration import *

""" 
Import model 
"""
sys.path.append( os.path.abspath('../../Network/model') )
from train import *
import model_out_5_heading_3_relu as MODEL
model_path = os.path.abspath('../../Network/notebook/train_log_20181208/model_out_1/model009.ckpt')

def NormalizeResample(data, num_sample):
    """ data is in N x ...
    we want to keep num_samplexC of them.
    if N > num_sample, we will randomly keep num_sample of them.
    if N < num_sample, we will randomly duplicate samples.
    """
  

    ## normalizing    
    min_val = np.min(data, axis = 0)
    
    data[:,0] = data[:,0] - min_val[0]
    data[:,1] = data[:,1] - min_val[1]
    data[:,2] = data[:,2] - min_val[2]
      
    max_val = np.max(data)
    
    data[:,0] = data[:,0] / max_val
    data[:,1] = data[:,1] / max_val
    data[:,2] = data[:,2] / max_val
                 
        
    ## resampling
    N = data.shape[0]
    if (N == num_sample):
        return data
    elif (N > num_sample):
        sample = np.random.choice(N, num_sample)
        return data[sample, ...]
    else:
        sample = np.random.choice(N, num_sample-N)
        dup_data = data[sample, ...]
        return np.concatenate([data, dup_data], 0)

for d in drives:
    
    """
    Load dataset
    """
    with open(d, 'rb') as f:
        parsed_kitti_data_w_r = pickle.load(f)

        """
        Classification
        """

        is_training = False

        GPU_INDEX = 0
        NUM_POINT = 1024
        NUM_CLASSES = 3
        BATCH_SIZE = 1

        # reset graph
        tf.reset_default_graph()

        with tf.device('/gpu:'+str(GPU_INDEX)):
            pointclouds_pl, labels_pl, heading_labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
            is_training_pl = tf.placeholder(tf.bool, shape=())

            # simple model
            pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)

            # Add ops to save and restore all the variables.
            saver = tf.train.Saver()

        # Create a session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        config.allow_soft_placement = True
        config.log_device_placement = True

        ops = {'pointclouds_pl': pointclouds_pl,
               'is_training_pl': is_training_pl,
               'pred': pred}

        # ops.reset_default_graph() 

        sess = tf.Session(config=config)

        # Restore variables from disk.
        saver.restore(sess, model_path)

        print "Model restored"
        sys.stdout.flush()


        for date in parsed_kitti_data_w_r:
            print "date:", date

            for drive in parsed_kitti_data_w_r[date]:
                print "\tdrive:", drive

                for i in range( len(parsed_kitti_data_w_r[date][drive]) ):

                    clusters = parsed_kitti_data_w_r[date][drive][i]['pc_wr']
                    parsed_kitti_data_w_r[date][drive][i]['cls_wr'] = []
                    for j in tqdm.trange(len(clusters)):

                        if len(clusters[j]) == 0:
                            continue

                        current_data = NormalizeResample(clusters[j], NUM_POINT)
                        feed_dict = {ops['pointclouds_pl']: current_data.reshape((BATCH_SIZE, NUM_POINT, 3)),
                                     ops['is_training_pl']: is_training}
                        pred_val = sess.run([ops['pred']], feed_dict=feed_dict )

                        cls = np.argmax(pred_val[0][:3], 1)
                        parsed_kitti_data_w_r[date][drive][i]['cls_wr'].append(cls)

                    clusters = parsed_kitti_data_w_r[date][drive][i]['pc']
                    parsed_kitti_data_w_r[date][drive][i]['cls'] = []
                    for j in tqdm.trange(len(clusters)):

                        if len(clusters[j]) == 0:
                            continue
                        current_data = NormalizeResample(clusters[j][:,:3], NUM_POINT)
                        feed_dict = {ops['pointclouds_pl']: current_data.reshape((BATCH_SIZE, NUM_POINT, 3)),
                                     ops['is_training_pl']: is_training}
                        pred_val = sess.run([ops['pred']], feed_dict=feed_dict )

                        cls = np.argmax(pred_val[0][:3], 1)
                        parsed_kitti_data_w_r[date][drive][i]['cls'].append(cls)
                    
    """
    Save data
    """
    out_data_path = 'parsed_kitti_data_w_r_c_181212_{}.pickle'.format(drive)
    
    with open(out_data_path, 'wb') as f:
        pickle.dump(parsed_kitti_data_w_r, f, protocol=2)
    
    print("\n\nSaved new dataset, protocol=2")
    
    
print "\n\n======================================================="
print "======================================================="        
print "Execution time: {} ".format( time.time() - start_time )
print "======================================================="
print "=======================================================\n\n"

INFO:tensorflow:Restoring parameters from /media/kyungpyo/SmartCarContest/MasterThesisProject/Network/notebook/train_log_20181208/model_out_1/model009.ckpt
Model restored


  0%|          | 0/1 [00:00<?, ?it/s]

date: 2011_09_26
	drive: 0002


100%|██████████| 16/16 [00:00<00:00, 26.01it/s]




Saved new dataset, protocol=2


Execution time: 32.107899189 


